In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-21 22:31:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-06-21 22:31:09 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [6]:
# Deliverable 2 (Step 1)
total_votes = df.filter(df["total_votes"]>20).select(["total_votes"])
total_votes.show()

+-----------+
|total_votes|
+-----------+
|         34|
|         35|
|        175|
|         31|
|         60|
|         65|
|         36|
|         34|
|         31|
|         32|
|        198|
|         51|
|         36|
|        139|
|         45|
|        116|
|         28|
|         23|
|        110|
|        119|
+-----------+
only showing top 20 rows



In [7]:
helpful_votes = df.select("helpful_votes")
helpful_votes.show()

+-------------+
|helpful_votes|
+-------------+
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            0|
|            1|
|            0|
|            0|
|            0|
|            0|
|            0|
+-------------+
only showing top 20 rows



In [8]:
new_votes = df.select(["total_votes", "helpful_votes"])
new_votes.show()

+-----------+-------------+
|total_votes|helpful_votes|
+-----------+-------------+
|          0|            0|
|          0|            0|
|          1|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          1|            1|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
|          0|            0|
+-----------+-------------+
only showing top 20 rows



In [9]:
# Deliverable 2 (Step 2)
newer_votes = new_votes.filter(new_votes["total_votes"]>=50)
newer_votes.show()

+-----------+-------------+
|total_votes|helpful_votes|
+-----------+-------------+
|        175|          147|
|         60|           55|
|         65|           51|
|        198|          151|
|         51|           49|
|        139|            7|
|        116|           34|
|        110|           88|
|        119|           99|
|        106|          102|
|         75|            8|
|         91|           79|
|         53|           44|
|         55|           44|
|        172|          150|
|         61|           49|
|         52|            3|
|        117|            6|
|         82|           79|
|        256|          243|
+-----------+-------------+
only showing top 20 rows



In [10]:
yes_vine_votes = newer_votes.filter(df["vine"] == 'Y')
yes_vine_votes.show()

+-----------+-------------+
|total_votes|helpful_votes|
+-----------+-------------+
|         63|           56|
|         51|           46|
|         58|           55|
|         58|           51|
|        103|           85|
|         50|           38|
|        262|          241|
|         75|           69|
|        107|           97|
|         53|           28|
|        340|          321|
|         86|           71|
|         62|           58|
|         54|           47|
|         62|           53|
|        126|          111|
|        143|          125|
|        110|           89|
|        357|          331|
|         88|           66|
+-----------+-------------+
only showing top 20 rows



In [11]:
no_vine_votes = newer_votes.filter(df["vine"] == 'N')
no_vine_votes.show()

+-----------+-------------+
|total_votes|helpful_votes|
+-----------+-------------+
|        175|          147|
|         60|           55|
|         65|           51|
|        198|          151|
|         51|           49|
|        139|            7|
|        116|           34|
|        110|           88|
|        119|           99|
|        106|          102|
|         75|            8|
|         91|           79|
|         53|           44|
|         55|           44|
|        172|          150|
|         61|           49|
|         52|            3|
|        117|            6|
|         82|           79|
|        256|          243|
+-----------+-------------+
only showing top 20 rows



In [12]:
#Total Number of Reviews
total_review_count = df.select("review_id")
total_review_count.count()

1785997

In [13]:
total_stars = df.select("star_rating")
total_stars.count()

1785997

In [16]:
# Total Number of 5 Star Reviews
total_5_stars = total_stars.filter(total_stars["star_rating"] == '5')
total_5_stars.count()

1026924

In [17]:
# Percentage of 5-star reviews (Yes Vine)
paid_comp = vine_df.filter(vine_df["vine"] == 'Y').filter(vine_df["star_rating"] == '5')
paid_comp.count()

1607

In [18]:
# Percentage of 5-star reviews (No Vine)
no_paid_comp = vine_df.filter(vine_df["vine"] == 'N').filter(vine_df["star_rating"] == '5')
no_paid_comp.count()

1025317

In [19]:
percentage_y = paid_comp.count() / total_5_stars.count()
percentage_y

0.0015648675072351995

In [20]:
percentage_n = no_paid_comp.count() / total_5_stars.count()
percentage_n

0.9984351324927648